In [1]:

from sklearn.ensemble import ExtraTreesRegressor
# from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import time
# from catboost import CatBoostRegressor, metrics

In [2]:
# Path to training data set
training_data_path = "../data/08_3_2022/data3/a_extract_regression_data_1n1p_tb.txt"
# Path to testing data set
testing_data_path = "../data/08_3_2022/data3/c_extract_regression_data_1n1p_hb.txt"
# Whether to normalize data or not
norm = True
# Name of the model
m_name = "ert"
# Whether to load an existing model
load = False

In [3]:

def create_ert():
    return ExtraTreesRegressor(n_estimators=100, n_jobs=-1, verbose=True)
    # return CatBoostRegressor(objective='MultiRMSE')

In [4]:
def skip_n_and_load_dataset(n,path):
    accumulator = []
    with open(path,'r') as f:
        for _ in range(n):
            next(f)
        for line in f:
            data = line.split(":")
            first_part = data[0].lstrip().split()
            second_part = data[1].lstrip().split()
            third_part = data[2].split("==>")[1].lstrip().split()
            extra_value = data[2].split("==>")[0].lstrip().split()[2]
            sec = first_part[0]
            charge = first_part[1]
            output = second_part
            input = third_part[:6]
            # accumulator.append([int(sec), int(charge), float(input[0]), float(input[1]), float(input[2]), float(input[3]), float(input[4]), float(input[5]), float(output[0]), float(output[1]), float(output[2]) , float(extra_value)])
            accumulator.append([int(sec), int(charge), float(input[0]), float(input[1]), float(input[2]), float(input[3]), float(input[4]), float(input[5]), float(output[0]), float(output[1]), float(output[2])])
            # print(line)
    return np.array(accumulator)

In [5]:
data = skip_n_and_load_dataset(0,training_data_path)

In [6]:
# Validate that the data look fine. Data layout:
print("Sector\tCharge\tIn[0]\tIn[1]\tIn[2]\tIn[3]\tIn[4]\tIn[5]\tOut[0]\tOut[1]\tOut[2]")  
print(data[:5])

Sector	Charge	In[0]	In[1]	In[2]	In[3]	In[4]	In[5]	Out[0]	Out[1]	Out[2]
[[   4.       -1.       18.571    17.571    18.833    17.857    23.833
    22.833     4.9017   10.3041  172.0683]
 [   3.        1.       46.667    43.       40.667    35.       23.
    18.        1.0985   20.7015   70.3843]
 [   3.       -1.       20.5      21.333    21.       21.833    26.4
    27.167     4.0101   11.0097  127.0826]
 [   1.        1.       60.5      64.5      56.       58.833    48.143
    50.167     2.0701   25.3714   -1.392 ]
 [   4.       -1.        4.833     5.333     8.167    10.       24.167
    26.        1.9716    6.3195 -169.1469]]


In [7]:
# Prepare datasets based on charge and filter sector 2

# Sector 2 data
sec_2_filter = data[:, 0] == 2
sec_2_data = data[sec_2_filter]

# Sector 2 pos charge
sec_2_pos_charge_filter = sec_2_data[:, 1] == 1
sec_2_pos_data = sec_2_data[sec_2_pos_charge_filter]

# Sector 2 neg charge
sec_2_neg_charge_filter = sec_2_data[:, 1] == -1
sec_2_neg_data = sec_2_data[sec_2_neg_charge_filter]

# Normalize X data
X_data_pos = sec_2_pos_data[:,2:8]/111
X_data_neg = sec_2_neg_data[:,2:8]/111

# Prepar y data
y_data_pos = sec_2_pos_data[:,8:]
y_data_neg = sec_2_neg_data[:,8:]
print(y_data_pos.shape)
# print(all_pos_data[:5])
# print(all_neg_data[:5])
# print(sec_2_data.shape)
# print(sec_2_pos_data[0])

(7316, 3)


In [8]:
# If training a new model prepare the scaler and train the network
# Otherwise load existing network and scaler

if load == False:
    clf_neg = create_ert()
    X_scaler_neg = StandardScaler()
    y_scaler_neg = StandardScaler()
    X_train_neg, X_test_neg, y_train_neg, y_test_neg =  train_test_split(X_data_neg, y_data_neg, test_size=0.05, random_state = 42)
    y_train_neg_norm = y_scaler_neg.fit_transform(y_train_neg)
    if norm:
        import pickle
        y_test_neg_norm = y_scaler_neg.transform(y_test_neg)
        clf_neg.fit(X_train_neg, y_train_neg_norm)
        with open(m_name+'_neg.model', 'wb') as file:
            pickle.dump(clf_neg, file)
        with open(m_name+'_neg.scaler', 'wb') as file:
            pickle.dump(y_train_neg_norm, file)
        score_neg = clf_neg.score(X_train_neg, y_train_neg_norm)
        print("Training score neg: ",score_neg)

    else:
        clf_neg.fit(X_train_neg, y_train_neg)
        score_neg = clf_neg.score(X_train_neg, y_train_neg)
        print("Training score neg: ",score_neg)
else:
    import pickle
    with open(m_name+'_neg.model', 'rb') as file:
        clf_neg = pickle.load(file)
    if norm:
        with open(m_name+'_neg.scaler', 'rb') as file:
            y_scaler_neg = pickle.load(file)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 80 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 out of 100 | elapsed:    1.4s remaining:    1.9s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    2.4s finished
[Parallel(n_jobs=80)]: Using backend ThreadingBackend with 80 concurrent workers.
[Parallel(n_jobs=80)]: Done  42 out of 100 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=80)]: Done 100 out of 100 | elapsed:    0.2s finished


Training score neg:  0.9999992622616881


In [9]:
# If training a new model prepare the scaler and train the network
# Otherwise load existing network and scaler

if load == False:
    clf_pos = create_ert()
    X_scaler_pos = StandardScaler()
    y_scaler_pos = StandardScaler()
    X_train_pos, X_test_pos, y_train_pos, y_test_pos =  train_test_split(X_data_pos, y_data_pos, test_size=0.05, random_state = 42)
    y_train_pos_norm = y_scaler_pos.fit_transform(y_train_pos)
    if norm:
        import pickle
        y_test_pos_norm = y_scaler_pos.transform(y_test_pos)
        clf_pos.fit(X_train_pos, y_train_pos_norm)
        with open(m_name+'_pos.model', 'wb') as file:
            pickle.dump(clf_pos, file)
        with open(m_name+'_pos.scaler', 'wb') as file:
            pickle.dump(y_train_pos_norm, file)
        score_pos = clf_pos.score(X_train_pos, y_train_pos_norm)
        print("Training score pos: ",score_pos)

    else:
        clf_pos.fit(X_train_pos, y_train_pos)
        score_pos = clf_pos.score(X_train_pos, y_train_pos)
        print("Training score pos: ",score_pos)
else:
    import pickle
    with open(m_name+'_pos.model', 'rb') as file:
        clf_pos = pickle.load(file)
    if norm:
        with open(m_name+'_pos.scaler', 'rb') as file:
            y_scaler_pos = pickle.load(file)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 80 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 out of 100 | elapsed:    1.6s remaining:    2.2s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    2.7s finished
[Parallel(n_jobs=80)]: Using backend ThreadingBackend with 80 concurrent workers.


Training score pos:  1.0


[Parallel(n_jobs=80)]: Done  42 out of 100 | elapsed:    0.2s remaining:    0.3s
[Parallel(n_jobs=80)]: Done 100 out of 100 | elapsed:    0.2s finished


In [10]:
# Evaluate on sector 2 validation set (pos)

if norm:
    score_pos = clf_pos.score(X_test_pos, y_test_pos_norm)
    print("Testing score_pos: ",score_pos)
else :
    score_pos = clf_pos.score(X_test_pos, y_test_pos)
    print("Testing score_pos: ",score_pos)

[Parallel(n_jobs=80)]: Using backend ThreadingBackend with 80 concurrent workers.
[Parallel(n_jobs=80)]: Done  42 out of 100 | elapsed:    0.2s remaining:    0.3s


Testing score_pos:  0.9776301273908015


[Parallel(n_jobs=80)]: Done 100 out of 100 | elapsed:    0.2s finished


In [11]:
# Evaluate on sector 2 validation set (neg)

if norm:
    score_neg = clf_neg.score(X_test_neg, y_test_neg_norm)
    print("Testing score_neg: ",score_neg)
else :
    score_neg = clf_neg.score(X_test_neg, y_test_neg)
    print("Testing score_neg: ",score_neg)

Testing score_neg:  0.9882631265895924


[Parallel(n_jobs=80)]: Using backend ThreadingBackend with 80 concurrent workers.
[Parallel(n_jobs=80)]: Done  42 out of 100 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=80)]: Done 100 out of 100 | elapsed:    0.1s finished


In [12]:
# Load testing data

data_read = skip_n_and_load_dataset(0, testing_data_path)


In [13]:
# Prepare testing data set based on charge and normalize with scaler

data = data_read
# sec_3_data = data[data[:, 0] == 3] 
# sec_2_data = data[data[:, 0] == 2] 
# sec_3_data[:, 10] -= 60
# data = sec_3_data
# data = np.concatenate((sec_3_data, sec_2_data))
all_pos_data_filter = data[:,1] == 1
all_neg_data_filter = data[:,1] == -1
all_pos_data = data[all_pos_data_filter]
all_neg_data = data[all_neg_data_filter]

X_all_pos = all_pos_data[:,2:8]/111
y_all_pos = all_pos_data[:, 8:]
y_all_pos_norm = y_scaler_pos.transform(y_all_pos)

X_all_neg = all_neg_data[:,2:8]/111
y_all_neg = all_neg_data[:, 8:]

y_all_neg_norm = y_scaler_neg.transform(y_all_neg)


In [14]:
print(X_all_pos.shape)
print(y_all_pos.shape)
print(data[:5])
print(all_pos_data[:5])
# pos_predict = clf_pos.predict(X_all_pos)

(67010, 6)
(67010, 3)
[[  5.      -1.      30.286   34.167   30.      34.      34.8     38.857
    2.7493  15.2538 -89.4344]
 [  3.       1.      74.5     75.      64.167   62.5     44.833   42.4
    0.7025  35.0741  71.5885]
 [  3.      -1.      13.667   15.      14.167   15.571   19.833   21.167
    5.0194   8.9759 130.2519]
 [  1.       1.      53.      54.167   44.857   43.143   22.5     19.444
    0.8124  24.512  -34.6423]
 [  3.      -1.      20.571   21.333   21.143   21.833   26.4     27.167
    4.063   11.4135 123.5638]]
[[  3.       1.      74.5     75.      64.167   62.5     44.833   42.4
    0.7025  35.0741  71.5885]
 [  1.       1.      53.      54.167   44.857   43.143   22.5     19.444
    0.8124  24.512  -34.6423]
 [  1.       1.      60.5     64.5     56.      58.833   48.143   50.167
    2.104   26.502    0.5452]
 [  3.       1.      60.143   67.      52.667   55.857   30.      29.375
    0.6113  33.2836  90.7207]
 [  4.       1.      58.167   64.      52.286   55.5  

In [15]:
# Evaluate on all sectors of testing data set (pos)

pos_predict = None
if norm:
    score_pos = clf_pos.score(X_all_pos, y_all_pos_norm)
    print("Score Positive Data: ", score_pos)
    pos_predict = y_scaler_pos.inverse_transform(clf_pos.predict(X_all_pos))
else:
    score_pos = clf_pos.score(X_all_pos, y_all_pos)
    print("Score Positive Data: ", score_pos)
    pos_predict = clf_pos.predict(X_all_pos)
# print(pos_predict[1])

[Parallel(n_jobs=80)]: Using backend ThreadingBackend with 80 concurrent workers.
[Parallel(n_jobs=80)]: Done  42 out of 100 | elapsed:    1.9s remaining:    2.6s
[Parallel(n_jobs=80)]: Done 100 out of 100 | elapsed:    2.1s finished
[Parallel(n_jobs=80)]: Using backend ThreadingBackend with 80 concurrent workers.


Score Positive Data:  0.6161449644252611


[Parallel(n_jobs=80)]: Done  42 out of 100 | elapsed:    1.8s remaining:    2.5s
[Parallel(n_jobs=80)]: Done 100 out of 100 | elapsed:    2.2s finished


In [16]:
# Evaluate on all sectors of testing data set (neg)

neg_predict = None
if norm:
    score_neg = clf_neg.score(X_all_neg, y_all_neg_norm)
    print("Score negitive Data: ", score_neg)
    start = time.clock()
    neg_predict = y_scaler_neg.inverse_transform(clf_neg.predict(X_all_neg))
    end = time.clock()
else:
    score_neg = clf_neg.score(X_all_neg, y_all_neg)
    print("Score Negative Data: ", score_neg)
    start = time.clock()
    neg_predict = clf_neg.predict(X_all_neg)
    end = time.clock()
print("Time {}".format(end-start))
print(neg_predict[1])

[Parallel(n_jobs=80)]: Using backend ThreadingBackend with 80 concurrent workers.
[Parallel(n_jobs=80)]: Done  42 out of 100 | elapsed:    1.5s remaining:    2.0s
[Parallel(n_jobs=80)]: Done 100 out of 100 | elapsed:    1.9s finished
/home/pthom001/anaconda3/envs/ml/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """
[Parallel(n_jobs=80)]: Using backend ThreadingBackend with 80 concurrent workers.


Score negitive Data:  0.47250186188511695


[Parallel(n_jobs=80)]: Done  42 out of 100 | elapsed:    1.8s remaining:    2.5s
[Parallel(n_jobs=80)]: Done 100 out of 100 | elapsed:    2.0s finished


Time 3.9883179999999996
[ 5.129381  8.949817 71.814066]


/home/pthom001/anaconda3/envs/ml/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  import sys


In [17]:
# Appends predictions to testing data set

def skip_n_and_append_predictions_to_file(n, path):
    index_pos = 0
    index_neg = 0
    with open(path, 'r') as f:
        suffix = '.pred'
        if norm :
            suffix += '.norm'
        with open(path+suffix, 'w') as out:
            for _ in range(n):
                out.write(f.readline())
            for line in f:
                data = line.split(":")
                first_part = data[0].lstrip().split()
                second_part = data[1].lstrip().split()
                third_part = data[2].split("==>")[1].lstrip().split()
                extra_value = data[2].split("==>")[0].lstrip().split()[2]
                sec = first_part[0]
                charge = first_part[1]
                output = second_part
                input = third_part[:6]
                if int(charge) == -1:
                    if y_all_neg[index_neg, 0]  == float(output[0]) and y_all_neg[index_neg, 1]  == float(output[1]) and y_all_neg[index_neg, 2]  == float(output[2]):
                    # if y_all_neg[index_neg, 0]  == float(extra_value):
                        out.write(line.strip() + "   {} {} {}\n".format(neg_predict[index_neg, 0], neg_predict[index_neg, 1], neg_predict[index_neg, 2]))
                        # out.write(line.strip() + ": {} \n".format(neg_predict[index_pos]))
                    else:
                        print(line)
                        print("ERROR!")
                        break
                    index_neg += 1
                elif int(charge) == 1:
                    if y_all_pos[index_pos, 0]  == float(output[0]) and y_all_pos[index_pos, 1]  == float(output[1]) and y_all_pos[index_pos, 2]  == float(output[2]):
                    # if y_all_pos[index_pos, 0]  == float(extra_value):
                        out.write(line.strip() + "   {} {} {}\n".format(pos_predict[index_pos, 0], pos_predict[index_pos, 1], pos_predict[index_pos, 2]))
                        # out.write(line.strip() + ": {} \n".format(pos_predict[index_pos]))
                    else:
                        print("ERROR!")
                        break
                    index_pos += 1

In [18]:
# Append predictions to testing set
skip_n_and_append_predictions_to_file(0, testing_data_path)

In [ ]:
print(X_all_neg.shape)